# 简介
非阻塞socket的I/O事件循环（使用水平触发的I/O多路复用），Python3上，tornado ioloop是对asyncio中ioloop的封装，
在python2时，使用epoll或者kqueue实现，如果都不可用，回退到select系统调用。

典型demo如下：

```python
import asyncio
import logging

import tornado.httpclient
import tornado.httpserver
import tornado.log
import tornado.options
import tornado.web


class AsyncHttpHandler(tornado.web.RequestHandler):
    def get(self):
        self.finish("hello")


def make_app():
    return tornado.web.Application(
        handlers=[(r"/", AsyncHttpHandler)],
        debug=True)


if __name__ == '__main__':
    tornado.options.parse_command_line()
    app = make_app()
    server = tornado.httpserver.HTTPServer(app, xheaders=True)
    server.listen(8888)
    tornado.ioloop.IOLoop.current().start()
```

# ioloop 实例



python2中，ioloop绑定到当前线程，python3中，委托给asyncio创建


```python

    # In Python 2, _current.instance points to the current IOLoop.
    _current = threading.local()

    # In Python 3, _ioloop_for_asyncio maps from asyncio loops to IOLoops.
    _ioloop_for_asyncio = dict()

    @staticmethod
    def current(instance=True):
        """Returns the current thread's `IOLoop`.

        If an `IOLoop` is currently running or has been marked as
        current by `make_current`, returns that instance.  If there is
        no current `IOLoop` and ``instance`` is true, creates one.

        .. versionchanged:: 4.1
           Added ``instance`` argument to control the fallback to
           `IOLoop.instance()`.
        .. versionchanged:: 5.0
           On Python 3, control of the current `IOLoop` is delegated
           to `asyncio`, with this and other methods as pass-through accessors.
           The ``instance`` argument now controls whether an `IOLoop`
           is created automatically when there is none, instead of
           whether we fall back to `IOLoop.instance()` (which is now
           an alias for this method). ``instance=False`` is deprecated,
           since even if we do not create an `IOLoop`, this method
           may initialize the asyncio loop.
        """
        if asyncio is None:
            current = getattr(IOLoop._current, "instance", None)
            if current is None and instance:
                current = IOLoop()
                if IOLoop._current.instance is not current:
                    raise RuntimeError("new IOLoop did not become current")
        else:
            try:
                loop = asyncio.get_event_loop()
            except (RuntimeError, AssertionError):
                if not instance:
                    return None
                raise
            try:
                return IOLoop._ioloop_for_asyncio[loop]
            except KeyError:
                if instance:
                    from tornado.platform.asyncio import AsyncIOMainLoop
                    current = AsyncIOMainLoop(make_current=True)
                else:
                    current = None
        return current
```